잠재요인기반 추천시스템 ALS 모델 만들기

In [1]:
from pyspark.sql import SparkSession

MAX_MEMORY = '5g'
spark = SparkSession.builder.appName("241212_01_MLlib_als")\
            .config("spark.executor.memory", MAX_MEMORY) \
            .config("spark.driver.memory", MAX_MEMORY) \
            .getOrCreate()

24/12/13 14:43:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Data 적재재

In [2]:
rating_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/ratings.csv')

In [3]:
rating_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [4]:
rating_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
# 전처리 - timestamp 제거

In [7]:
rating_df = rating_df.select(['userId','movieId', 'rating'])


In [54]:
rating_df.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
+------+-------+------+
only showing top 5 rows



데이터 세트 분할

In [8]:
train_ratio = 0.8
test_ratio = 0.2

train_df, test_df = rating_df.randomSplit([train_ratio, test_ratio], seed=42)

ALS모델 객체 생성

In [22]:
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=5,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

모델의 학습

In [23]:
als_model = als.fit(train_df)

예측 확인

In [55]:
# userid, movieid > rating
predictions = als_model.transform(test_df)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 26480|    148|   2.0|  2.020703|
|151614|    148|   1.0|  2.744473|
| 28229|    148|   1.0| 2.2856345|
|  6491|    148|   4.0| 2.4144835|
| 14831|    148|   3.0| 2.7400835|
+------+-------+------+----------+
only showing top 5 rows



In [56]:
predictions.select("rating", "prediction").describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4998109|           4998109|
|   mean|3.5341648211353536|3.3784115385483697|
| stddev|1.0609230261741123|0.6329935917588668|
|    min|               0.5|        -1.9028906|
|    max|               5.0|         6.5752754|
+-------+------------------+------------------+



ALS 모델의 하이퍼파라미터 조정
coldStartStrategy = 'drop' > 예측에 충분하지 않은컬럼의 데이터의 경우 'drop' 
학습한 모델의 속성을 변경 > 다시 예측을 한 경우에도 조정이 안됨 > 학슥부터 다시 실행

In [57]:
als.setColdStartStrategy('drop')

ALS_f346f2d0c882

In [58]:
# userid, movieid > rating
predictions = als_model.transform(test_df)
predictions.show(5)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
| 26480|    148|   2.0|  2.020703|
|151614|    148|   1.0|  2.744473|
| 28229|    148|   1.0| 2.2856345|
|  6491|    148|   4.0| 2.4144835|
| 14831|    148|   3.0| 2.7400835|
+------+-------+------+----------+
only showing top 5 rows



In [59]:
predictions.select("rating", "prediction").describe().show()

+-------+------------------+------------------+
|summary|            rating|        prediction|
+-------+------------------+------------------+
|  count|           4998109|           4998109|
|   mean|3.5341648211353536|3.3784115385483697|
| stddev|1.0609230261741123|0.6329935917588668|
|    min|               0.5|        -1.9028906|
|    max|               5.0|         6.5752754|
+-------+------------------+------------------+



평가 RMSE 측정

In [31]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [32]:
rmse = evaluator.evaluate(predictions)
rmse

0.8199798867876938

In [ ]:
# 활용

1. 사용자id > 추천목록
2. 영화id > 사용자목록

In [ ]:
als_model.getRecommentForAllUsers(3)  #user별로 추천할 영화 3개를 골라준다.

In [ ]:
als_model.recommandForAllItems(3) #item별로 추천할 유저 3명을 골라준다

In [60]:
from pyspark.sql.types import IntegerType

In [61]:
user_list = [65, 78, 81]
user_df = spark.createDataFrame(user_list, IntegerType()).toDF("userId")
user_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [62]:
user_recommend_movies = als_model.recommendForUserSubset(user_df,3)
user_recommend_movies.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    65|[{205277, 7.68428...|
|    78|[{177209, 6.95740...|
|    81|[{179707, 5.03770...|
+------+--------------------+



In [36]:
# movie- list Load

In [40]:
movie_file = 'data/movies.csv'
movies_df = spark.read.csv(movie_file, inferSchema=True, header=True)

In [41]:
movies_df.show(3)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



In [42]:
#65번 사용자의 추천영화목록

movies_list = user_recommend_movies.collect()[0].recommendations
movies_list

[Row(movieId=205277, rating=7.6842803955078125),
 Row(movieId=194434, rating=7.081778049468994),
 Row(movieId=203086, rating=6.9282331466674805)]

In [63]:
rec_df = spark.createDataFrame(movies_list)
rec_df.show()

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 205277|7.6842803955078125|
| 194434| 7.081778049468994|
| 203086|6.9282331466674805|
+-------+------------------+



In [44]:
#영화 정보와 추천 영화 목록을 조인

In [64]:
rec_df.createOrReplaceTempView('recommend')
movies_df.createOrReplaceTempView('movies')

In [65]:
query = '''
select *
from movies join recommend on movies.movieId = recommend.movieId
order by rating desc
'''

In [66]:
recommend_movies = spark.sql(query)

In [67]:
recommend_movies.show()

+-------+--------------------+--------------------+-------+------------------+
|movieId|               title|              genres|movieId|            rating|
+-------+--------------------+--------------------+-------+------------------+
| 205277|   Inside Out (1991)|Comedy|Drama|Romance| 205277|7.6842803955078125|
| 194434|   Adrenaline (1990)|  (no genres listed)| 194434| 7.081778049468994|
| 203086|Truth and Justice...|               Drama| 203086|6.9282331466674805|
+-------+--------------------+--------------------+-------+------------------+



In [ ]:
#ALS 추천모델 > 튜닝

In [68]:
spark.stop()